# Gnomix Demo

This notebook takes you through the process of using the tool for local ancestry predictions both by training a model using your own reference panel and by using a pre-trained model. It then gives an example of how one might analyze the results.

### Loading packages

In [1]:
import numpy as np
import os
import pandas as pd

In [2]:
# Arguments
data_path        = "./demo/data/"
query_file       = data_path + "ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz"
genetic_map_file = data_path + "allchrs.b37.gmap"
reference_file   = data_path + "reference_1000g.vcf"
sample_map_file  = data_path + "1000g.smap"
chm              = "22"
phase            = "False"
output_basename  = "./demo/output"

## Training a model from scratch

Here we use the command line interface to train a model using a reference file, make inference on a query file and then write the results to the file *demo.msp.txv*. 

All the files are stored inside the */demo/data* folder **except for the query file and the reference file**. The query file we'll use is chromosome 22 from the 1000 genomes project which can be downloaded from ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/. We'll create the reference file from the query file in the next cell by subsetting the part of the query file that we know are single ancestry samples using the provided sample map (1000g.smap) with bcftools.

In [3]:
sample_map = pd.read_csv(sample_map_file, sep="\t")
samples = list(sample_map["#Sample"])
sample_file = data_path + "samples_1000g.tsv"
np.savetxt(sample_file, samples, delimiter="\t", fmt="%s")
subset_cmd = "bcftools view" + " -S " + sample_file + " -o " + reference_file + " " + query_file
print("Running in command line: \n\t", subset_cmd)
os.system(subset_cmd)

Running in command line: 
	 bcftools view -S ./demo/data/samples_1000g.tsv -o ./demo/data/reference_1000g.vcf ./demo/data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz


0

Now we have everything we need to train the model,

In [ ]:
train_cmd = " ".join(["python3 gnomix.py",
                      query_file, genetic_map_file, output_basename, chm, phase, reference_file, sample_map_file]) + \
            " > ./demo/training_log.txt"
print("Running in command line: \n\t", train_cmd)
os.system(train_cmd)

Running in command line: 
	 python3 gnomix.py ./demo/data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz ./demo/data/allchrs.b37.gmap ./demo/output 22 False ./demo/data/reference_1000g.vcf ./demo/data/1000g.smap > ./demo/training_log.txt


## Using a pre-trained model

The inference has already been done but we can repeat it using the stored model from the training. The model *./demo/model_chm_22.pkl.gz"* is the same as would be the result of the above process.

In [ ]:
path_to_model    = "./demo/model_chm_22.pkl.gz" 

# defining and executing the command
run_cmd =  "python3 gnomix.py"
cmd = " ".join([run_cmd, query_file, genetic_map_file, output_basename, chm, phase, path_to_model]) + \
      " > ./demo/pretrained_log.txt"
print("Running in command line: \n\t", cmd)
os.system(cmd)

## Analyzing results

### Reading from the ouput

Here we read the results from the output file into a dataframe

In [ ]:
# read the output file into a dataframe
output_file = output_basename+"/query_results.msp.tsv"
msp_df = pd.read_csv(output_file, sep="\t", skiprows=[0])
msp_df

## Measuring performance

In [ ]:
from scipy import stats
from sklearn.metrics import confusion_matrix

from src.postprocess import get_samples_from_msp_df
from src.visualization import plot_cm, plot_chm

%load_ext autoreload
%autoreload 2

In [ ]:
sample_map = pd.read_csv(sample_map_file, sep="\t")

Here we predict **single ancestry** for each individual by creatoing a dataframe containing only one label prediction for each individual. We do so simply by taking the mode of the local predictions for each individual. 

In [ ]:
# reading the sample-ids
query_samples = get_samples_from_msp_df(msp_df)

# reading predictions for each of the intervals
pred_labels = (np.array(msp_df)[:,6:].T).astype(int)

# reducing it to only 1 of maternal/paternal prediction
single_ind_idx = np.arange(0,len(query_samples)*2,2)
pred_labels_single_ind = pred_labels[single_ind_idx,:]

# predicting single ancestry by taking mode of local predictions for each individual
y_pred = stats.mode(pred_labels_single_ind,axis=1)[0].squeeze() 

# get model population order from first line of file and convert from numeric predictions
with open(output_file, "r") as f:
    pop = np.array([p.split("=")[0] for p in f.readline().split()[2:]])
pred_pop = [pop[pop_ind] for pop_ind in y_pred]

# put it together in dataframe
pred_df = pd.DataFrame({"Sample": query_samples, "Prediction": pred_pop})
pred_df

We can read the true labels from the sample map file and add them to the dataframe. The true ancestry labels (not only single ancestry individuals) are stored in the meta data file *demo/data/1000g.meta*.

In [ ]:
# adding the true labels
sample_map_file = "./demo/data/1000g.meta" # find the true labels in the sample map file
sample_map_df = pd.read_csv(sample_map_file, sep="\t")
true_labels_df = sample_map_df[["Sample", "Superpopulation code"]]
true_labels_df.columns = ["Sample", "Population"]
pred_df = pred_df.merge(true_labels_df)
pred_df

And then compare the predictions with the true labels

In [ ]:
# Getting the prediction accuracy
acc = np.mean(pred_df.Population == pred_df.Prediction)
print("Accuracy for single ancestry: ", acc*100, "%", sep="")

In [ ]:
# creating and visualizing the confusion matrix
labs = np.unique(pred_df[['Population', 'Prediction']])
cm = confusion_matrix(pred_df.Population, pred_df.Prediction, labels=labs)
cm_plot = plot_cm(cm, normalize=True, labels=labs)
cm_plot.figure.savefig('./demo/imgs/single_ancestry_confusion_matrix.png')

## Plotting the prediction

We can use the results to plot the predictions along the chromosome. Here is an example of how to use [Tagore](https://pypi.org/project/tagore/#usage) for that purpose. Here we visualize one individual and only for chromosome 22. See **plot_chm** from *src/visualization.py* for more details of how to to that.

In [ ]:
from IPython.display import Image

sample_id = "HG00096" # looking at just one random sample
img_name = "./demo/imgs/chm_img"
plot_chm(sample_id, msp_df, img_name=img_name)
Image(filename=img_name+".png")